<a href="https://colab.research.google.com/github/alexk2206/tds_capstone/blob/main/answer_combinations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import json

In [27]:

import pandas as pd

dfs = []

for i in range(1, 6):
    url = f"https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/questionnaires/questionnaire{i}.json"
    df = pd.read_json(url)
    df['options'] = df['options'].apply(lambda x: ", ".join([opt['option'] for opt in x]))
    dfs.append(df)

all_questions_df = pd.concat(dfs, ignore_index=True)
all_questions_df

,id,type,question,options
0,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,SINGLE_SELECT,Data processing consent,"Yes, No"
1,12e1ed1d-edaa-4e93-8645-de3850e998f9,SINGLE_SELECT,Customer group,"End User, Wholesaler, Distributor, Consultant,..."
2,625012ae-9192-4cf6-a73d-55e1813d6014,MULTI_SELECT,Products interested in,"MY-SYSTEM, Notion, JTS, JS EcoLine, AKW100, AX100"
3,0699fc5a-34a4-4160-bda1-fb135a3615da,MULTI_SELECT,What kind of follow up is planned,"Email, Phone, Schedule a Visit, No action"
4,815dab84-bc5e-4764-9777-0c0126e3173e,MULTI_SELECT,Who to copy in follow up,"Stephan Maier, Joachim Wagner, Erik Schneider,..."
5,3f34e5b3-1cb0-48ea-93d2-3f21b3371b5d,SINGLE_SELECT,Would you like to receive marketing informatio...,"Yes, No"
6,ba042f33-697e-4c6f-924c-b4de2c30f443,SINGLE_SELECT,What industry are you operating in?,"Aerospace, Computers & Networks, Government, M..."
7,7a776cc0-ffe8-4891-b8a9-dd5ff984de13,MULTI_SELECT,What products are you interested in?,"Automotive radar target simulation, Noise figu..."
8,a0148bc7-15b3-41d5-b97c-6420b8bd927c,TEXT,Notes,Please provide any additional information that...
9,5aefc81d-c5d2-41fc-bc7b-6117d1c7671e,SINGLE_SELECT,What type of company is it?,"Construction company, Craft enterprises, Scaff..."


In [28]:
filtered_df = all_questions_df#[(all_questions_df['type'] == 'MULTI_SELECT') | (all_questions_df['type'] == 'SINGLE_SELECT')]

filtered_df

,id,type,question,options
0,aa2d8cdd-0758-4035-b0b6-ca18e2f380d8,SINGLE_SELECT,Data processing consent,"Yes, No"
1,12e1ed1d-edaa-4e93-8645-de3850e998f9,SINGLE_SELECT,Customer group,"End User, Wholesaler, Distributor, Consultant,..."
2,625012ae-9192-4cf6-a73d-55e1813d6014,MULTI_SELECT,Products interested in,"MY-SYSTEM, Notion, JTS, JS EcoLine, AKW100, AX100"
3,0699fc5a-34a4-4160-bda1-fb135a3615da,MULTI_SELECT,What kind of follow up is planned,"Email, Phone, Schedule a Visit, No action"
4,815dab84-bc5e-4764-9777-0c0126e3173e,MULTI_SELECT,Who to copy in follow up,"Stephan Maier, Joachim Wagner, Erik Schneider,..."
5,3f34e5b3-1cb0-48ea-93d2-3f21b3371b5d,SINGLE_SELECT,Would you like to receive marketing informatio...,"Yes, No"
6,ba042f33-697e-4c6f-924c-b4de2c30f443,SINGLE_SELECT,What industry are you operating in?,"Aerospace, Computers & Networks, Government, M..."
7,7a776cc0-ffe8-4891-b8a9-dd5ff984de13,MULTI_SELECT,What products are you interested in?,"Automotive radar target simulation, Noise figu..."
8,a0148bc7-15b3-41d5-b97c-6420b8bd927c,TEXT,Notes,Please provide any additional information that...
9,5aefc81d-c5d2-41fc-bc7b-6117d1c7671e,SINGLE_SELECT,What type of company is it?,"Construction company, Craft enterprises, Scaff..."


In [29]:
from itertools import chain, combinations

def generate_combinations_with_empty(options_list):
    return list(chain.from_iterable(combinations(options_list, r) for r in range(0, len(options_list) + 1)))

filtered_df['options'] = filtered_df['options'].str.split(', ')

expanded_rows = []

# Iteriere über alle Zeilen im DataFrame
for _, row in filtered_df.iterrows():
    question = row['question']
    options_list = row['options']
    question_type = row['type']

    if question_type == 'MULTI_SELECT':
        options_combinations = generate_combinations_with_empty(options_list)
        for combo in options_combinations:
            expanded_rows.append({'question': question, 'type': question_type, 'options': list(combo)})
    elif question_type == 'SINGLE_SELECT':
        for option in options_list:
            expanded_rows.append({'question': question, 'type': question_type, 'options': [option]})
    else:
        expanded_rows.append({'question': question, 'type': question_type, 'options': options_list})

expanded_df = pd.DataFrame(expanded_rows)

expanded_df

,question,type,options
0,Data processing consent,SINGLE_SELECT,[Yes]
1,Data processing consent,SINGLE_SELECT,[No]
2,Customer group,SINGLE_SELECT,[End User]
3,Customer group,SINGLE_SELECT,[Wholesaler]
4,Customer group,SINGLE_SELECT,[Distributor]
...,...,...,...
8534,Searches a solution for,MULTI_SELECT,"[Clean up CRM, Extract data from emails, Impro..."
8535,Searches a solution for,MULTI_SELECT,"[Scan business cards, Clean up CRM, Extract da..."
8536,Next steps,SINGLE_SELECT,[Offer]
8537,Next steps,SINGLE_SELECT,[Meeting]


In [30]:
# count number of types
expanded_df['type'].value_counts()

,count
type,
MULTI_SELECT,8472
SINGLE_SELECT,63
TEXT,2
DATE,1
NUMBER,1
